In [18]:
from pipeline import *
from model_generator import get_3_log_model



paths = get_all_paths("/Users/noah/Flatiron/NN/AudioRecognition/Pickles/")

data_list,labels_list = list_of_pickle_paths_to_data_list_and_labels(paths)
data,labels = concate_multiprocess_outputs(data_list,labels_list)


In [20]:
from multifileprocess import data_label_write
data_label_write(data,labels,"./saved_all_pkl")

In [28]:
print(data.keys())

dict_keys(['log_S', 'log_Sh', 'log_Sp', 'C', 'mfcc', 'delta_mfcc', 'delta2_mfcc'])


In [2]:
data["log_S"].shape

(1000, 128, 433)

In [3]:
res = data["log_S"].reshape(tuple(list(data["log_S"].shape)+[1]))

In [4]:
res[330][3][5]

array([-73.27483909])

In [5]:
data["log_S"][330][3][5]

-73.27483908553316

In [6]:
res = data["log_S"].reshape(55424000)

In [12]:
set([resi is None for resi in res])

{False}

In [25]:
model = get_3_log_model()

In [2]:
keys=["log_S","log_Sh","log_Sp"]

In [16]:
labels[0] = "10"
labels[1] = '101'
labels[2] = '11'
labels[3] = '12'
labels[4] = '15'


In [26]:

fit_dict(data,labels,model,keys,epochs=1)

Epoch 1/1
5200/5200 [==============================] - 9900s 2s/step - loss: 12.5294 - acc: 0.2196


In [13]:
data["log_S"].shape

(1000, 128, 433)

In [8]:
input_vals = [reshape_for_model(data[key]) for key in keys]
labs = to_categorical(LabelEncoder().fit_transform(labels))

In [4]:
input_vals[0].shape

(1000, 128, 433, 1)

In [9]:
labs.shape

(1000, 1)

In [14]:
set([l[0] for l in labs])

{1.0, 10.0}

In [45]:
from keras import Model,Input
from keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,concatenate
from keras.regularizers import l2
from keras.optimizers import Adam


def log_attach(input_layer):
    x = Conv2D(128,kernel_size=4,padding="valid",data_format="channels_last",activation="relu",use_bias=True)(input_layer)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Conv2D(64,kernel_size=4,padding="valid",data_format="channels_last",activation="relu",use_bias=True)(x)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Conv2D(5,kernel_size = 3,padding = "valid",data_format = "channels_last",activation = "relu",use_bias=True)(x)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Flatten()(x)
    outputs = Dense(50,activation="relu",use_bias=True)(x)
    return Model(inputs=input_layer,outputs=outputs)

def get_log_model(num_of_log_transforms):
    inputs = [Input((128,433,1)) for i in range(num_of_log_transforms)]
    mods = [log_attach(inputs[i]) for i in range(num_of_log_transforms)]
    skip_first = False
    if num_of_log_transforms == 1:
        skip_first = True
        x = mods[0].output
        inputs = inputs[0]
    else:
        x = concatenate([mod.output for mod in mods])
    if not skip_first:
        x = Dense(100,activation="relu",use_bias=True)(com)
    x = Dense(50,activation="relu",use_bias=True)(x)
    x = Dense(5,activation="softmax",use_bias=True)(x)
    mod = Model(inputs = inputs,outputs = x)
    opt = Adam(lr=.01)
    mod.compile(optimizer=opt,loss='categorical_crossentropy',metrics=["accuracy"])
    return mod


In [46]:
sing_model = get_log_model(1)

In [47]:
sing_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 128, 433, 1)       0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 125, 430, 128)     2176      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 62, 215, 128)      0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 59, 212, 64)       131136    
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 29, 106, 64)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 27, 104, 5)        2885      
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 13, 52, 5)         0         
__________

In [51]:
keys=["log_S"]
data_sing = data["log_S"]
fit_dict(data_sing,labels,sing_model,keys,epochs=1)

Epoch 1/1
5200/5200 [==============================] - 2101s 404ms/step - loss: 13.1053 - acc: 0.1869


In [50]:
def fit_dict(data,labels,model,keys,**kwargs):
    '''
    To be used after the data has been processed.
    Fits the model to the processed data.

    Parameters:
        data
            the data dictionary produced by multifileprocess
        labels
            the label list produced by multifileprocess
        model
            Keras multiinput model
        keys
            the ordered list of keys for specifying which
            data features to pass to the model
        **kwargs
            will be passed to model.fit()
            Ex/ epochs = 100
    '''
    if type(data) != dict:
        input_vals = reshape_for_model(data)
    else:
        input_vals = [reshape_for_model(data[key]) for key in keys]
    labs = to_categorical(LabelEncoder().fit_transform(labels))
    return model.fit(x=input_vals,y=labs,**kwargs)

In [53]:
data["mfcc"].shape

(5200, 13, 433)

In [61]:
from keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,concatenate,Dropout
def get_mfcc_model():
    input_layer = Input((13,433,1))
    x = Conv2D(128,kernel_size=(4,3),padding="valid",data_format="channels_last",activation="relu",use_bias=True)(input_layer)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Dropout(.5)(x)
    x = Conv2D(32,kernel_size=3,padding="valid",data_format="channels_last",activation="relu",use_bias=True)(x)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(100,activation="relu",use_bias=True)(x)
    x = Dense(50,activation="relu",use_bias=True)(x)
    outputs = Dense(5,activation="softmax",use_bias=True)(x)
    mod = Model(inputs=input_layer,outputs=outputs)
    opt = Adam(lr=.0001)
    mod.compile(optimizer=opt,loss='categorical_crossentropy',metrics=["accuracy"])
    return mod


In [62]:
mfcc_model = get_mfcc_model()
keys=["mfcc"]
data_sing = data["mfcc"]
fit_dict(data_sing,labels,mfcc_model,keys,epochs=10)

Epoch 1/10
5200/5200 [==============================] - 86s 17ms/step - loss: 8.0558 - acc: 0.2260
Epoch 2/10
5200/5200 [==============================] - 83s 16ms/step - loss: 2.4963 - acc: 0.3035
Epoch 3/10
5200/5200 [==============================] - 82s 16ms/step - loss: 1.5222 - acc: 0.4171
Epoch 4/10
5200/5200 [==============================] - 84s 16ms/step - loss: 1.2124 - acc: 0.4963
Epoch 5/10
5200/5200 [==============================] - 91s 18ms/step - loss: 1.0495 - acc: 0.5531
Epoch 6/10
5200/5200 [==============================] - 82s 16ms/step - loss: 0.8927 - acc: 0.6248
Epoch 7/10
5200/5200 [==============================] - 82s 16ms/step - loss: 0.7620 - acc: 0.6779
Epoch 8/10
5200/5200 [==============================] - 85s 16ms/step - loss: 0.6573 - acc: 0.7256
Epoch 9/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.5694 - acc: 0.7619
Epoch 10/10
5200/5200 [==============================] - 82s 16ms/step - loss: 0.4928 - acc: 0.7904


In [63]:
fit_dict(data_sing,labels,mfcc_model,keys,epochs=10)

Epoch 1/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.4236 - acc: 0.8279
Epoch 2/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.3783 - acc: 0.8569
Epoch 3/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.3302 - acc: 0.8740
Epoch 4/10
5200/5200 [==============================] - 82s 16ms/step - loss: 0.2867 - acc: 0.8971
Epoch 5/10
5200/5200 [==============================] - 85s 16ms/step - loss: 0.2634 - acc: 0.9017
Epoch 6/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.2315 - acc: 0.9158
Epoch 7/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.2078 - acc: 0.9240
Epoch 8/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.1862 - acc: 0.9308
Epoch 9/10
5200/5200 [==============================] - 86s 16ms/step - loss: 0.1699 - acc: 0.9356
Epoch 10/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.1482 - acc: 0.9446


In [64]:
fit_dict(data_sing,labels,mfcc_model,keys,epochs=10)

Epoch 1/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.1423 - acc: 0.9473
Epoch 2/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.1312 - acc: 0.9517
Epoch 3/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.1081 - acc: 0.9592
Epoch 4/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.1006 - acc: 0.9637
Epoch 5/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.0939 - acc: 0.9658
Epoch 6/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.0754 - acc: 0.9746
Epoch 7/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.0658 - acc: 0.9773
Epoch 8/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.0621 - acc: 0.9802
Epoch 9/10
5200/5200 [==============================] - 88s 17ms/step - loss: 0.0534 - acc: 0.9806
Epoch 10/10
5200/5200 [==============================] - 92s 18ms/step - loss: 0.0512 - acc: 0.9817


In [65]:
fit_dict(data_sing,labels,mfcc_model,keys,epochs=10)

Epoch 1/10
5200/5200 [==============================] - 92s 18ms/step - loss: 0.0431 - acc: 0.9850
Epoch 2/10
5200/5200 [==============================] - 90s 17ms/step - loss: 0.0443 - acc: 0.9833
Epoch 3/10
5200/5200 [==============================] - 99s 19ms/step - loss: 0.0417 - acc: 0.9858
Epoch 4/10
5200/5200 [==============================] - 95s 18ms/step - loss: 0.0270 - acc: 0.9910
Epoch 5/10
5200/5200 [==============================] - 82s 16ms/step - loss: 0.0252 - acc: 0.9931
Epoch 6/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.0286 - acc: 0.9908
Epoch 7/10
5200/5200 [==============================] - 85s 16ms/step - loss: 0.0183 - acc: 0.9954
Epoch 8/10
5200/5200 [==============================] - 84s 16ms/step - loss: 0.0194 - acc: 0.9937
Epoch 9/10
5200/5200 [==============================] - 89s 17ms/step - loss: 0.0190 - acc: 0.9948
Epoch 10/10
5200/5200 [==============================] - 93s 18ms/step - loss: 0.0130 - acc: 0.9977


In [66]:
fit_dict(data_sing,labels,mfcc_model,keys,epochs=10)

Epoch 1/10
5200/5200 [==============================] - 87s 17ms/step - loss: 0.0125 - acc: 0.9960
Epoch 2/10
5200/5200 [==============================] - 86s 17ms/step - loss: 0.0108 - acc: 0.9975
Epoch 3/10
5200/5200 [==============================] - 86s 17ms/step - loss: 0.0122 - acc: 0.9962
Epoch 4/10
5200/5200 [==============================] - 88s 17ms/step - loss: 0.0112 - acc: 0.9967
Epoch 5/10
5200/5200 [==============================] - 90s 17ms/step - loss: 0.0145 - acc: 0.9952
Epoch 6/10
5200/5200 [==============================] - 105s 20ms/step - loss: 0.0126 - acc: 0.9965
Epoch 7/10
5200/5200 [==============================] - 89s 17ms/step - loss: 0.0087 - acc: 0.9979
Epoch 8/10
5200/5200 [==============================] - 86s 17ms/step - loss: 0.0164 - acc: 0.9942
Epoch 9/10
5200/5200 [==============================] - 83s 16ms/step - loss: 0.0079 - acc: 0.9979
Epoch 10/10
5200/5200 [==============================] - 82s 16ms/step - loss: 0.0066 - acc: 0.9985


In [67]:
mfcc_model.save("wellfitmfccmodel")

In [68]:
keras.load("wellfitmfccmodel")

NameError: name 'keras' is not defined

In [69]:
from keras.models import load_model
da_model = load_model("wellfitmfccmodel")

In [70]:
da_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 13, 433, 1)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 10, 431, 128)      1664      
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 5, 215, 128)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 215, 128)       0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 3, 213, 32)        36896     
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 1, 106, 32)        0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 3392)              0         
__________

In [74]:
predict = data["mfcc"]

In [75]:
data["mfcc"].shape

(5200, 13, 433)

In [77]:
predict = predict.reshape((5200, 13, 433,1))

In [78]:
predict.shape

(5200, 13, 433, 1)

In [80]:
predictions = da_model.predict(predict)

In [81]:
predictions.shape

(5200, 5)

In [83]:
predictions[1]

array([1.4275165e-21, 1.5475767e-27, 6.3356688e-06, 3.3310320e-31,
       9.9999368e-01], dtype=float32)

In [84]:
def greatest(array):
    current_best = -1
    current_best_ind = -1
    for i in range(len(array)):
        if array[i]> current_best:
            current_best = array[i]
            current_best_ind = i
    return i

In [82]:
labels

['tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',
 'tr',

In [85]:
pred_nums = [greatest(arr) for arr in predictions]

In [86]:
pred_nums

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [87]:
set(pred_nums)

{4}

In [88]:
set(labels)

{'ar', 'fa', 'gk', 'ru', 'tr'}